<a href="https://colab.research.google.com/github/mariacmartins/bioinformatica-disciplina/blob/main/Aula_5_Aprendizado_de_M%C3%A1quina_%E2%80%93_Aprendizado_Supervisionado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 5: Aprendizado de Máquina – Aprendizado Supervisionado

- **Inteligência Artificial**: É um subárea da Ciência da Computação, dedicada à mimetização de processos que caracterizam algum nível de inteligência.
- **Aprendizado de Máquina**: Área da IA dedicada à replicação do processo de aprendizagem. 
- **Aprendizado Supervisionado**: Consiste no aprendizado a partir de conjuntos de dados "rotulados", onde já sabemos qual é a saída correta (visto que temos rótulos no conjunto de dados) e tendo a ideia de que existe uma relação entrada-saída.
- **Treinamento de Modelo:** No caso do aprendizado supervisionado, o treinamento consiste em apresentarmos para o algoritmo um número suficiente de exemplos de entradas e saídas desejadas (já rotuladas previamente). 

### PhageProtDetector 
Imagine que você está trabalhando em um projeto de análise de vírus bacteriófagos e agora precisa criar uma ferramenta preditiva para discriminar proteínas de bactérias de proteínas de fagos. Para isso, você resolve utilizar dados de proteínas disponíveis no UniProt e em um primeiro momento testar se a composição de aminoácidos é suficiente para discriminar as proteínas de bactérias e fagos.

1. Baixe um conjunto de proteínas de bactérias (bacteria.fasta) e fagos (phage.fasta) do UniProt 

In [1]:
!wget -O phage.fasta    'http://bit.ly/bioinfo-ml-phage'
!wget -O bacteria.fasta 'http://bit.ly/bioinfo-ml-bacteria'

--2021-04-12 12:30:09--  http://bit.ly/bioinfo-ml-phage
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.uniprot.org/uniprot/?query=taxonomy:phage&sort=score&limit=10000&format=fasta [following]
--2021-04-12 12:30:09--  https://www.uniprot.org/uniprot/?query=taxonomy:phage&sort=score&limit=10000&format=fasta
Resolving www.uniprot.org (www.uniprot.org)... 128.175.245.202, 193.62.193.81
Connecting to www.uniprot.org (www.uniprot.org)|128.175.245.202|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [text/plain]
Saving to: ‘phage.fasta’

phage.fasta             [      <=>           ]   5.21M  4.89MB/s    in 1.1s    

2021-04-12 12:30:11 (4.89 MB/s) - ‘phage.fasta’ saved [5465789]

--2021-04-12 12:30:11--  http://bit.ly/bioinfo-ml-bacteria
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connect

2. Construa um DataFrame do Pandas contendo a composição de aminoácidos destas proteínas e o campo de rótulo “phage”, que deve ser igual a 1 quando a proteína for de fago e igual a 0 quando for de bactéria (pode utilizar a função compute_aa_composition vista em aula e a mesma estrutura lógica para preencher o DataFrame – fazendo-se os devidos ajustes). 

In [3]:
!pip install biopython

     |████████████████████████████████| 2.3MB 5.9MB/s 


In [4]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam
import pandas as pd

In [5]:
def compute_aa_composition(protein_sequence):
  protein_analyzer = ProtParam.ProteinAnalysis(str(protein_sequence))
  protein_data = protein_analyzer.get_amino_acids_percent()
  return protein_data

In [6]:
df = pd.DataFrame()

handle = open('phage.fasta')
parser = SeqIO.parse(handle, 'fasta')

for record in parser:
  protein_data = compute_aa_composition(record.seq)
  protein_data['phage'] = 1
  df = df.append(protein_data, ignore_index=True)


handle = open('bacteria.fasta')
parser = SeqIO.parse(handle, 'fasta')

for record in parser:
  protein_data = compute_aa_composition(record.seq)
  protein_data['phage'] = 0
  df = df.append(protein_data, ignore_index=True)

df.head(10)

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,phage
0,0.034857,0.004648,0.062742,0.054996,0.055771,0.046476,0.007746,0.099148,0.065066,0.072812,0.018590,0.116189,0.034082,0.026336,0.039504,0.075910,0.061967,0.054222,0.010844,0.058095,1.0
1,0.028213,0.007053,0.063480,0.056426,0.050940,0.040752,0.009404,0.097179,0.074451,0.085423,0.014890,0.098746,0.036834,0.033699,0.030564,0.090125,0.057210,0.057994,0.013323,0.053292,1.0
2,0.027972,0.010490,0.055944,0.024476,0.024476,0.013986,0.006993,0.080420,0.055944,0.094406,0.013986,0.153846,0.010490,0.076923,0.031469,0.097902,0.066434,0.052448,0.034965,0.066434,1.0
3,0.053968,0.015873,0.085714,0.047619,0.028571,0.069841,0.025397,0.073016,0.057143,0.076190,0.028571,0.057143,0.041270,0.031746,0.041270,0.047619,0.066667,0.126984,0.003175,0.022222,1.0
4,0.061856,0.006873,0.082474,0.082474,0.054983,0.061856,0.024055,0.072165,0.072165,0.096220,0.020619,0.041237,0.020619,0.030928,0.054983,0.054983,0.054983,0.054983,0.010309,0.041237,1.0
5,0.086598,0.006186,0.063918,0.080412,0.047423,0.068041,0.024742,0.045361,0.041237,0.094845,0.020619,0.028866,0.043299,0.030928,0.078351,0.072165,0.035052,0.070103,0.020619,0.041237,1.0
6,0.102436,0.008596,0.057307,0.047994,0.027221,0.102436,0.013610,0.058739,0.053009,0.062321,0.020057,0.070201,0.031519,0.029370,0.042980,0.106734,0.070917,0.055874,0.013610,0.025072,1.0
7,0.027972,0.010490,0.055944,0.024476,0.024476,0.013986,0.006993,0.080420,0.055944,0.094406,0.013986,0.153846,0.010490,0.076923,0.031469,0.097902,0.066434,0.052448,0.034965,0.066434,1.0
8,0.035313,0.006421,0.046549,0.052970,0.040128,0.048154,0.001605,0.104334,0.052970,0.073836,0.008026,0.128411,0.043339,0.033708,0.033708,0.086677,0.081862,0.048154,0.001605,0.072231,1.0
9,0.057377,0.014754,0.080328,0.062295,0.040984,0.055738,0.034426,0.080328,0.070492,0.072131,0.029508,0.037705,0.034426,0.034426,0.039344,0.077049,0.049180,0.065574,0.022951,0.040984,1.0


3. Cria as variáveis X, y, contendo, respectivamente, os dados de composição de aminoácido e os rótulos das proteínas baixadas.
4.	Separe os dados de treino dos dados de validação com a função train_test_split. O resultado deve ser salvo nas variáveis X_train, X_test, y_train e y_test. 


In [7]:
X = df.drop(['phage'], axis=1)
y = df['phage']

In [8]:
X

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,0.034857,0.004648,0.062742,0.054996,0.055771,0.046476,0.007746,0.099148,0.065066,0.072812,0.018590,0.116189,0.034082,0.026336,0.039504,0.075910,0.061967,0.054222,0.010844,0.058095
1,0.028213,0.007053,0.063480,0.056426,0.050940,0.040752,0.009404,0.097179,0.074451,0.085423,0.014890,0.098746,0.036834,0.033699,0.030564,0.090125,0.057210,0.057994,0.013323,0.053292
2,0.027972,0.010490,0.055944,0.024476,0.024476,0.013986,0.006993,0.080420,0.055944,0.094406,0.013986,0.153846,0.010490,0.076923,0.031469,0.097902,0.066434,0.052448,0.034965,0.066434
3,0.053968,0.015873,0.085714,0.047619,0.028571,0.069841,0.025397,0.073016,0.057143,0.076190,0.028571,0.057143,0.041270,0.031746,0.041270,0.047619,0.066667,0.126984,0.003175,0.022222
4,0.061856,0.006873,0.082474,0.082474,0.054983,0.061856,0.024055,0.072165,0.072165,0.096220,0.020619,0.041237,0.020619,0.030928,0.054983,0.054983,0.054983,0.054983,0.010309,0.041237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.178082,0.017613,0.052838,0.035225,0.021526,0.097847,0.027397,0.046967,0.017613,0.121331,0.017613,0.039139,0.050881,0.043053,0.056751,0.056751,0.043053,0.058708,0.007828,0.009785
19996,0.225681,0.005837,0.066148,0.019455,0.023346,0.120623,0.025292,0.025292,0.007782,0.116732,0.013619,0.015564,0.068093,0.015564,0.058366,0.029183,0.066148,0.071984,0.011673,0.013619
19997,0.217899,0.005837,0.062257,0.019455,0.019455,0.124514,0.023346,0.027237,0.009728,0.120623,0.013619,0.015564,0.071984,0.015564,0.060311,0.019455,0.068093,0.079767,0.009728,0.015564
19998,0.180361,0.008016,0.050100,0.038076,0.018036,0.116232,0.028056,0.038076,0.022044,0.136273,0.016032,0.020040,0.044088,0.046092,0.050100,0.050100,0.044088,0.070140,0.014028,0.010020


In [9]:
y

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
19995    0.0
19996    0.0
19997    0.0
19998    0.0
19999    0.0
Name: phage, Length: 20000, dtype: float64

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y)

5. Crie um modelo preditivo do tipo RandomForest e o treine com os dados de treinamento (X_train e y_train).
6. Valide o modelo com os dados de validação (X_test e y_test).
7. Gere um relatório de classificação para o modelo produzido. 
8. Crie uma célula de texto descrevendo a performance do modelo.


In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [12]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         0.0       0.92      0.96      0.94      2480
         1.0       0.96      0.92      0.94      2520

    accuracy                           0.94      5000
   macro avg       0.94      0.94      0.94      5000
weighted avg       0.94      0.94      0.94      5000



**Random forests** é um algoritmo de aprendizado supervisionado, que pode ser usado tanto para classificação quanto para regressão. Uma "floresta" (forest) é composta por árvores. De modo geral, quanto mais árvores, mais robusta é a floresta. Random forests criam árvores de decisão em amostras de dados selecionadas aleatoriamente, obtém previsões de cada árvore e seleciona a melhor solução. 

**Métricas de avaliação:**
- **Acurácia:** indica uma performance geral do modelo. Dentre todas as classificações, quantas o modelo classificou corretamente.
- **Precisão: **dentre todas as classificações de classe positivo que o modelo fez, quantas estão corretas.
- **Recall:** dentre todas as situações de classe positivo como valor esperado, quantas estão corretas.
- **F1-Score:** média harmônica entre precisão e recall.

**Classificação:** 
- Proteínas de fagos: 1
- Proteínas de bactéria: 0

**Conclusão geral:**
O modelo apresentou ótimos valores nas métricas de avaliação e consequentemente boa perfomance, com um bom valor de acurácia e f1-score **(0.94)**.